In [ ]:
import sys  
sys.path.insert(1, '/media/andrew/One Touch/HMNET2_crf0')

import numpy as np
import matplotlib.pyplot as plt
import os

# Copyright (c) Prophesee S.A.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
# http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express
# or implied. See the License for the specific language governing
# permissions and limitations under the License.

# This file is modified from the original code at
# https://github.com/prophesee-ai/prophesee-automotive-dataset-toolbox/blob/master/src/psee_evaluator.py
# The list of modifications are as follows:
# (1) "min_box_side" for box filtering is modified following the previous work:
#     Perot, Etienne, et al. "Learning to detect objects with a 1 megapixel event camera." Advances in Neural Information Processing Systems 33 (2020): 16639-16652.
# (2) Configs for GEN1 and GEN4 are added and passed to "evaluate_detection"

import glob
import numpy as np
import os
import argparse
import pickle as pkl
from numpy.lib import recfunctions as rfn

from coco_eval import evaluate_detection
from hmnet.utils.psee_toolbox.io.box_filtering import filter_boxes
from hmnet.utils.psee_toolbox.io.box_loading import reformat_boxes
from hmnet.utils.common import get_list, mkdir

EVAL_CONF_GEN1 = dict(
    classes = ('car', 'pedestrian'),
    width = 304,
    height = 240,
    time_tol = 25000, # +/- 25 msec (50 msec)
)

EVAL_CONF_GEN4 = dict(
    classes = ('pedestrian', 'two wheeler', 'car', 'truck', 'bus', 'traffic sign', 'traffic light'),
    width = 1280,
    height = 720,
    time_tol = 25000, # +/- 25 msec (50 msec)
)

def evaluate_folders(dt_folder, gt_lst, camera):
    dt_file_paths = get_list(dt_folder, ext='npy')
    gt_file_paths = get_list(gt_lst, ext='npy')

    filtered_paths = []
    for path in gt_file_paths:
    # Check if any reference path is a substring of the current path
        if any(ref.split('/')[-1].rsplit("_", 1)[0] in path for ref in dt_file_paths):
            filtered_paths.append(path)

    gt_file_paths = filtered_paths

    # assert len(dt_file_paths) == len(gt_file_paths)
    print("There are {} GT bboxes and {} PRED bboxes".format(len(gt_file_paths), len(dt_file_paths)))
    result_boxes_list = [np.load(p) for p in dt_file_paths]
    gt_boxes_list = [np.load(p) for p in gt_file_paths]
    gt_boxes_list = [ rfn.drop_fields(p, 'invalid') for p in gt_boxes_list ]

    result_boxes_list = [reformat_boxes(p) for p in result_boxes_list]
    gt_boxes_list = [reformat_boxes(p) for p in gt_boxes_list]

    min_box_diag = 60 if camera == 'GEN4' else 30
    min_box_side = 20 if camera == 'GEN4' else 10
    eval_conf = EVAL_CONF_GEN4 if camera == 'GEN4' else EVAL_CONF_GEN1

    filter_boxes_fn = lambda x:filter_boxes(x, int(5e5), min_box_diag, min_box_side)

    gt_boxes_list = map(filter_boxes_fn, gt_boxes_list)
    result_boxes_list = map(filter_boxes_fn, result_boxes_list)
    coco_eval = evaluate_detection(gt_boxes_list, result_boxes_list, **eval_conf)
    # print(coco_eval.curve_df.to_string())
    
    # Plot the PR curve
    return coco_eval.curve_df_small, coco_eval.curve_df_medium, coco_eval.curve_df_large, coco_eval.curve_df
    

gt_dir = "/media/andrew/One Touch/HMNET2_crf9_shrink/experiments/detection/data/gen1/test_lbl/"




In [ ]:
def plotem(adder_dir, shrink_dir):

    results_dir = "/media/andrew/One Touch/HMNET2_crf9/experiments/detection/workspace/hmnet_B3_yolox_tbptt/result/pred_test"
    small, medium, large, alll = evaluate_folders(adder_dir, gt_dir, 'GEN1')
    
    results_dir = "/media/andrew/One Touch/HMNET2_crf9_shrink/experiments/detection/workspace/hmnet_B3_yolox_tbptt/result/pred_test"
    small_s, medium_s, large_s, alll_s = evaluate_folders(shrink_dir, gt_dir, 'GEN1')
    
    plt.plot(small['threshold'], small['precision'])
    plt.plot(small_s['threshold'], small_s['precision'])
    plt.show()
    plt.plot(medium['threshold'], medium['precision'])
    plt.plot(medium_s['threshold'], medium_s['precision'])
    plt.show()
    plt.plot(large['threshold'], large['precision'])
    plt.plot(large_s['threshold'], large_s['precision'])
    plt.show()
    plt.plot(alll['threshold'], alll['precision'])
    plt.plot(alll_s['threshold'], alll_s['precision'])
    plt.show()



In [ ]:
plotem("/media/andrew/One Touch/HMNET2_crf9/experiments/detection/workspace/hmnet_B3_yolox_tbptt/result/pred_test", "/media/andrew/One Touch/HMNET2_crf9_shrink/experiments/detection/workspace/hmnet_B3_yolox_tbptt/result/pred_test")

In [ ]:
plotem("/media/andrew/One Touch/HMNET2_crf6/experiments/detection/workspace/hmnet_B3_yolox_tbptt/result/pred_test", "/media/andrew/One Touch/HMNET2_crf6_shrink/experiments/detection/workspace/hmnet_B3_yolox_tbptt/result/pred_test")

In [ ]:
plotem("/media/andrew/One Touch/HMNET2_crf3/experiments/detection/workspace/hmnet_B3_yolox_tbptt/result/pred_test", "/media/andrew/One Touch/HMNET2_crf3_shrink/experiments/detection/workspace/hmnet_B3_yolox_tbptt/result/pred_test")

In [ ]:
plotem("/media/andrew/One Touch/HMNET2_crf0/experiments/detection/workspace/hmnet_B3_yolox_tbptt/result/pred_test", "/media/andrew/One Touch/HMNET2_crf0_shrink/experiments/detection/workspace/hmnet_B3_yolox_tbptt/result/pred_test")